# Avance 1: Proyecto

## Entendimiento y preparación de los datos

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

In [2]:
# Ruta del archivo
file_path = "Datos_proyecto.xlsx"

# Cargar el Excel
df = pd.read_excel(file_path, sheet_name="Sheet1")

# Mostrar las primeras filas
df.head()

,textos,labels
0,"""Aprendizaje"" y ""educación"" se consideran sinó...",4
1,Para los niños más pequeños (bebés y niños peq...,4
2,"Además, la formación de especialistas en medic...",3
3,En los países de la OCDE se tiende a pasar de ...,4
4,Este grupo se centró en las personas que padec...,3


In [3]:
# Ver información general
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2424 entries, 0 to 2423
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   textos  2424 non-null   object
 1   labels  2424 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 38.0+ KB


Separamos nuestros conjuntos de entrenamiento y prueba

In [4]:
# Separar train/test
X_train, X_test, y_train, y_test = train_test_split(
    df["textos"], df["labels"], test_size=0.2, random_state=42
)

Ahora empezaremos construyendo el pipeline de preprocesamiento y entrenamiento de nuestros datos. Pero antes revisaremos TfidfVectorizer que nos ayudará a preprocesar nuestras entradas y a construir nuestra bag of words (BOW).

In [6]:
import unicodedata
import nltk
from nltk.corpus import stopwords

# Se preparan las stopwords en español para utilizar en vectorizer
nltk.download('stopwords')

# Se quitan los acentos de las stopwords, lo cual también es importante (evitar warnings)
def strip_accents(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFKD', s)
                   if not unicodedata.combining(c))


spanish_stopwords = sorted({ strip_accents(w.lower()) for w in stopwords.words('spanish') })
len(spanish_stopwords), spanish_stopwords[:20]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\incar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(306,
 ['a',
  'al',
  'algo',
  'algunas',
  'algunos',
  'ante',
  'antes',
  'como',
  'con',
  'contra',
  'cual',
  'cuando',
  'de',
  'del',
  'desde',
  'donde',
  'durante',
  'e',
  'el',
  'ella'])

In [7]:
# Usamos solo una muestra pequeña de tus datos para verlos "a mano"
muestra = df["textos"].head(5)

# Crear el vectorizador
vectorizer = TfidfVectorizer(
            stop_words=spanish_stopwords,   # stop words en español
            lowercase=True,                 # minúsculas
            strip_accents='unicode',      # quitar los acentos: educación -> educacion
            min_df=2,                       # ignora términos que aparezcan en <2 docs
            max_df=0.9,                     # ignora términos muy frecuentes >90%
            ngram_range=(1,2),              # se incluyen bigramas
            sublinear_tf=True,
            max_features=20000,           # evita sobrecargas por bigramas
)

# Ajustar y transformar
X_tfidf = vectorizer.fit_transform(muestra)

# Ver las palabras del vocabulario que creó TF-IDF
print("Palabras en el vocabulario:\n", vectorizer.get_feature_names_out())

Palabras en el vocabulario:
 ['cada' 'cada vez' 'forma' 'formación' 'mental' 'mentales' 'más' 'nivel'
 'ocde' 'pueden' 'salud' 'salud mental' 'servicios' 'servicios salud'
 'trastornos' 'trastornos mentales' 'tratamiento' 'vez' 'vez más']


## Modelado y Evaluación

Ahora construiremos los pipelines para el preprocesamiento y entrenamiento de nuestros datos de prueba

In [8]:
tfidf = TfidfVectorizer(
    stop_words=sorted(spanish_stopwords),
    lowercase=True,
    strip_accents='unicode',
    min_df=2,          # ignora términos muy raros
    max_df=0.9,        # ignora términos demasiado frecuentes
    ngram_range=(1,2), # unigrams + bigrams
    sublinear_tf=True, # tf = 1 + log(tf) AJUSTE PARA EL CONTEO, MEJOR LOGARITMICO
    max_features=20000 # límite para no explotar RAM
)


**Pipeline 1 (Regresión Logística)**

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Pipeline: TF-IDF + Logistic Regression
pipe_logreg = Pipeline([
    ("tfidf", tfidf),
    ("clf", LogisticRegression(
        max_iter=2000,
        class_weight="balanced",  # maneja desbalance de clases
        multi_class="auto"
    ))
])

pipe_logreg.fit(X_train, y_train)
y_pred_logreg = pipe_logreg.predict(X_test)

print("=== Regresión Logística ===")
print(classification_report(y_test, y_pred_logreg, digits=3))


C:\Users\incar\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


=== Regresión Logística ===
              precision    recall  f1-score   support

           1      0.956     0.964     0.960       112
           3      0.971     0.982     0.976       168
           4      0.995     0.980     0.988       205

    accuracy                          0.977       485
   macro avg      0.974     0.976     0.975       485
weighted avg      0.978     0.977     0.977       485



**Pipeline 2 (SVM Máquinas de Vectores de Soporte)**

In [10]:
from sklearn.svm import LinearSVC

# Pipeline: TF-IDF + Linear SVM
pipe_svm = Pipeline([
    ("tfidf", tfidf),
    ("clf", LinearSVC(class_weight="balanced", dual=True))
])

pipe_svm.fit(X_train, y_train)
y_pred_svm = pipe_svm.predict(X_test)

print("=== Support Vector Machine (Lineal) ===")
print(classification_report(y_test, y_pred_svm, digits=3))

=== Support Vector Machine (Lineal) ===
              precision    recall  f1-score   support

           1      0.956     0.964     0.960       112
           3      0.965     0.982     0.973       168
           4      0.995     0.976     0.985       205

    accuracy                          0.975       485
   macro avg      0.972     0.974     0.973       485
weighted avg      0.976     0.975     0.975       485



**Pipeline 3 (Bayes Ingenuo Multinomial)**

In [11]:
from sklearn.naive_bayes import MultinomialNB

# Pipeline: TF-IDF + Multinomial Naive Bayes
pipe_nb = Pipeline([
    ("tfidf", tfidf),
    ("clf", MultinomialNB(alpha=0.5))  # alpha suaviza probabilidades (HIPERPARAMETRO?)
])

pipe_nb.fit(X_train, y_train)
y_pred_nb = pipe_nb.predict(X_test)

print("=== Naive Bayes Multinomial ===")
print(classification_report(y_test, y_pred_nb, digits=3))


=== Naive Bayes Multinomial ===
              precision    recall  f1-score   support

           1      0.969     0.830     0.894       112
           3      0.943     0.976     0.959       168
           4      0.940     0.985     0.962       205

    accuracy                          0.946       485
   macro avg      0.950     0.931     0.938       485
weighted avg      0.947     0.946     0.945       485

